## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

3574


In [3]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
979


In [5]:
print('Input total sales volume ($$$ million) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million) from previous (not last) month:
849000000


In [6]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
435000


In [7]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
442


## Data Clean

In [8]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [9]:
df = df.dropna(subset=['SOLD DATE'])

In [10]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: May ---- Month before report month: April


In [11]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [12]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    1079
Name: count, dtype: int64
-------
$/SQUARE FEET
False    1066
True       13
Name: count, dtype: int64
-------
YEAR BUILT
False    1079
Name: count, dtype: int64
-------


In [13]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Sunny-Isles-Beach/17975-Collins-Ave-33160/unit-1502/home/185472178


In [14]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [15]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [16]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [17]:
# df_filtered = df_filtered[df_filtered['ADDRESS'] != '1901 Brickell Ave Unit B109']

In [59]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS'],
      dtype='object')

In [64]:
df_filtered.sort_values(by='$/SQUARE FEET',ascending=False).head(20)['URL'].iloc[1]

'https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-2203/home/43284562'

In [21]:
print(df_filtered['URL'].iloc[559])

https://www.redfin.com/FL/Miami/150-SE-25th-Rd-33129/unit-12I/home/42759453


In [22]:
# Correct the prices, if needed
df_filtered.at[576,'PRICE']=(310_000)
df_filtered.at[561,'PRICE']=(428_000)
df_filtered.at[559,'PRICE']=(495_000)


# Correct the psf, if needed
df_filtered.at[576,'$/SQUARE FEET']=(310_000/1117)
df_filtered.at[561,'$/SQUARE FEET']=(428_000/990)
df_filtered.at[559,'$/SQUARE FEET']=(495_000/804)

In [21]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [23]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
611,110000.0,1800 NE 114th St #607,Miami,75.0
599,155000.0,1800 NE 114th St #2206,Miami,79.0
565,75000.0,13480 NE 6th Ave #116,North Miami,88.0
584,129000.0,1750 NE 191st St Unit 414-2,Miami,125.0
608,175000.0,1780 NE 191st St Unit 200-2,North Miami Beach,133.0
189,250000.0,11111 Biscayne Blvd Unit 10B,Miami,138.0
600,110000.0,165 NE 203rd Ter #15,Miami Gardens,146.0
597,175000.0,8001 Crespi Blvd Unit 3B,Miami Beach,152.0
606,170000.0,14840 Naranja Lakes Blvd Unit C3M,Homestead,158.0
198,250000.0,18061 Biscayne Blvd #304,Aventura,160.0


In [23]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [24]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [25]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [26]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [27]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [28]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [29]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [30]:
m.save('index.html')

## Summary Info

In [31]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [32]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [33]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [34]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/1523-West-Ave-33139/unit-302/home/42800130


In [35]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
CONTINUUM ON SOUTH BEACH, 100 S Pointe Dr #2203 Miami Beach | Price $17,800,000 | $3,727 psf | Year built: 2002
Least Expensive
ZURICH CONDO, 13480 NE 6th Ave #116 North Miami | Price $75,000 | $88 psf | Year built: 1974
Highest Price Per Square Foot
CONTINUUM ON SOUTH BEACH, 50 S Pointe Dr #3401 Miami Beach | Price $14,425,000 | $4,972 psf | Year built: 2008
Lowest Price Per Square Foot
CRICKET CLUB CONDO, 1800 NE 114th St #607 Miami | Price $110,000 | $75 psf | Year built: 1975
Newest
PERRINE MANOR, 11275 SW 189th Ter #11275 Miami | Price $409,000 | $nan psf | Year built: 2024
Oldest
1523 WEST CONDO, 1523 West Ave #302 Miami Beach | Price $289,000 | $525 psf | Year built: 1924


## Map URL Snagger

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_may_2024


## Get Summary Data

In [37]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 989
--------
Total sale price: $779,608,945
--------
Median sale price: $410,000
--------
Max sale price: $17,800,000
--------
Min sale price: $75,000
------------------------------------------------
PSF INFO
Max price per square foot: $4,972
--------
Min price per square foot: $75
--------
Median price per square foot: $444
------------------------------------------------
CONDO AGES
Newest building: 2024.0
----------
Oldest building: 1924.0
----------
Average building age: 1987.7765419615773


In [38]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

# GENERATE STORY TEMPLATE

# STOP! DID YOU UPDATE VALUES FROM THE PREVIOUS MONTH?

In [39]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    social = 'higher'
    seo = 'increases'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    social = 'lower'
    seo = 'decreases'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    social = 'equal'
    seo = "doesn't change"
      
if subject_month_closings > prior_month_closings:
    dek_rf = 'rose'
    graf = 'up'
elif prior_month_closings > subject_month_closings:
    dek_rf ='falls'
    graf = 'down'
else:
    dek_rf = 'equals'
    graf = 'flat'

In [40]:
subject_month_closings

989

In [41]:
prior_month_sales_volume

849000000

In [42]:
prior_month_closings

979

In [43]:
story_string = f'''
\033[1mHED:\033[0m Miami-Dade County condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales volume {hed_rf} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m Miami-Dade County had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m


Miami-Dade County’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {graf} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max()/1_000_000:,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max()/1_000_000:,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. WHEN XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [44]:
# print(df_filtered['URL'].iloc[0])

In [45]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")


HED: Miami-Dade County condo sales dollar volume drops in May to $780M 
DEK: Number of closings rose to 989 from 979 in April
FEATURED HED:
SEO HED: Miami-Dade County May Condo Sales Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales volume drops to $780 million in May.
AUTHOR: Adam Farence
RESEARCH: 
Social: Miami-Dade County had $780 million in condo sales in May, lower than $849 million from April, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, monthly condo sales, condos, condo market, Continuum On South Beach, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:


Miami-Dade County’s NEWS PEG HERE.

May condo sales totaled $780 million, falling from $849 million in April. Brokers closed 989 sales last month, up from 979 sales in April and XXX in XXX, Multiple Listing Service data from Red

# Print links for notable sales

## Top sale

In [46]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-2203/home/43284562


## Cheapest sale

In [47]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/North-Miami/13480-NE-6th-Ave-33161/unit-116/home/42878604


## Highest PSF

In [48]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami-Beach/50-S-Pointe-Dr-33139/unit-3401/home/43391843


## Lowest PSF

In [49]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/1800-NE-114th-St-33181/unit-607/home/43379334


## Newest

In [50]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami/11275-SW-189th-Ter-33157/unit-11275/home/188373705


## Oldest

In [51]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/1523-West-Ave-33139/unit-302/home/42800130


## Time on Market Calculator

In [56]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 10, 31) ## List (Earlier) date
date2 = datetime(2024, 5, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

203
